# Блокнот для генерации деревьев траектрий, чтобы потом из них можно было сделать хорошую БД для обучения

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from missile_gym import MissileGym
from gymtree import GymTree, Node
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

%matplotlib widget
# %config InlineBackend.figure_format = 'svg'

### Генерируем стандартный сценарий

In [3]:
MissileGym.scenario_names

{'Chengdu_1',
 'Chengdu_2',
 'Cy-57_1',
 'RaptorF_1',
 'RaptorF_2',
 'Refale_1',
 'Refale_2',
 'fail_1',
 'fail_2',
 'fail_3',
 'sc_simple_1',
 'sc_simple_2',
 'sc_simple_3',
 'standart',
 'success_1',
 'success_2',
 'success_3',
 'success_4',
 'success_5'}

In [4]:
gym = MissileGym.make('fail_3')
tree = GymTree(gym)
tree.reset()

In [5]:
# выращиваем дерево
def f(x, d0=900):
    t = (x-d0/3)/abs(d0-d0/3)
    return t
for i in tqdm(range(5)):
    node = tree.root # np.random.choice(tree.get_not_full_nodes()) 
    d0 = node.get_distance_to_trg()
    while node:
        d_curr = node.get_distance_to_trg()
        p = f(d_curr, d0)
        node = node.walk(0)
for i in tqdm(range(5)):
    node = tree.get_perspective_node2()
    d0 = node.get_distance_to_trg()
    while node:
        d_curr = node.get_distance_to_trg()
        p = f(d_curr, d0)
        node = node.walk(p)

In [6]:
tree.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
tree.plot_scatter()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig, ax = plt.subplots(figsize=(10,7))
tree.plot_scatter(ax=ax)
ax.grid()
ax.axis('equal')
plt.savefig('saves/sc_simple_3.png', format='png', dpi=300);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# сохраняем
tree.save_to_file('saves/sc_simple_3.bin')

## А теперь посчитаем их все.... параллельно

In [10]:
from dask.distributed import Client, LocalCluster

In [11]:
client = Client(LocalCluster())
client

Client Scheduler: tcp://127.0.0.1:63670 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.11 GB


In [22]:
def grow_tree(scenario_name):
    try:
        import numpy as np
        from missile_gym import MissileGym
        from gymtree import GymTree, Node
        import os
        
        def f(x, d0=900):
            t = (x-d0/3)/abs(d0-d0/3)
            return t

        gym = MissileGym.make(scenario_name)
        tree = GymTree(gym)
        tree.reset()
        
        file_name = f'saves/{scenario_name}.bin'
        if os.path.isfile(file_name):
            tree.load_from_file(file_name)
            
        for i in range(1):
            node = tree.root # 
            d0 = node.get_distance_to_trg()
            while node:
                d_curr = node.get_distance_to_trg()
                node = node.walk(0)
       
        for i in range(177):
            node = tree.root # np.random.choice(tree.get_not_full_nodes()) #
            d0 = node.get_distance_to_trg()
            while node:
                d_curr = node.get_distance_to_trg()
                p = f(d_curr, d0)
                node = node.walk(p)
        for i in range(1777):
            node = tree.get_perspective_node2()
            d0 = node.get_distance_to_trg()
            while node:
                d_curr = node.get_distance_to_trg()
                p = f(d_curr, d0)
                node = node.walk(p)
        
        tree.save_to_file(file_name)
        
        import matplotlib.pyplot as plt
        
        fig, ax = plt.subplots(figsize=(10,7))
        tree.plot_scatter(ax=ax)
        ax.grid()
        ax.axis('equal')
        plt.savefig(f'saves/{scenario_name}.png', format='png', dpi=300)
        return True
    except Exception as e:
        return str(e)

In [23]:
futs = client.map(grow_tree, MissileGym.scenario_names)

In [21]:
futs

[<Future: finished, type: builtins.bool, key: grow_tree-9fff39bcdf22bfef365e56e4bfc374fb>,
 <Future: finished, type: builtins.bool, key: grow_tree-83abc59e6960a67dff0947ad7912ea62>,
 <Future: finished, type: builtins.bool, key: grow_tree-3933d3fd9ac9dd5170a2dbd7da01a7af>,
 <Future: finished, type: builtins.bool, key: grow_tree-43cc5977100ee30a6e398071ccf3619c>,
 <Future: finished, type: builtins.bool, key: grow_tree-cec77f1d4f4b4156b3742ef43c0763b2>,
 <Future: finished, type: builtins.bool, key: grow_tree-46ac4cffe8d3895d6eab2d536ec627cb>,
 <Future: finished, type: builtins.str, key: grow_tree-9fec3ccce8488110655fd193f91c6912>,
 <Future: finished, type: builtins.bool, key: grow_tree-8f80630ec49b4096d1418f676be1d2df>,
 <Future: finished, type: builtins.bool, key: grow_tree-87f47049ff3b3e358178958ea7316eae>,
 <Future: finished, type: builtins.bool, key: grow_tree-57a14418091b0969a9e2bd6e3adc8dca>,
 <Future: finished, type: builtins.bool, key: grow_tree-af64f8888f170ebd41c47f953d2f1eb4>,


In [20]:
np.random.permutation(10)

array([1, 6, 4, 5, 7, 3, 8, 0, 9, 2])

In [ ]:
np.